# NER with spaCy
**"Named Entity Recognition"** is a subtask of NLP where we extract specific named entities from the text. The definition of a "named entity" changes depending on the domain we're working on. We'll look at clinical NER later, but first we'll look at some examples in more general domains.

NER is often performed using news articles as source texts. In this case, named entities are typically proper nouns, such as:
- People
- Geopolitical entities, like countries
- Organizations

We won't go into the details of how NER is implemented in spaCy. If you want to learn more about NER and various way it's implemented, a great resource is [Chapter 17.1 of Jurafsky and Martin's textbook "Speech and Language Processing."](https://web.stanford.edu/~jurafsky/slp3/17.pdf)

In [ ]:
import spacy
from spacy import displacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

Here is an excerpt from an article in the Guardian. We'll process this document with our nlp object and then look at what entities are extracted. One way to do this is using spaCy's `displacy` package, which visualizes the results of a spaCy pipeline.

In [ ]:
text = """Germany will fight to the last hour to prevent the UK crashing out of the EU without a deal and is willing 
to hear any fresh ideas for the Irish border backstop, the country’s ambassador to the UK has said.
Speaking at a car manufacturers’ summit in London, Peter Wittig said Germany cherished its relationship 
with the UK and was ready to talk about solutions the new prime minister might have for the Irish border problem."""

In [ ]:
doc = nlp(text)

In [ ]:
displacy.render(doc, style="ent")

We can use spaCy's `explain` function to see definitions of what an entity type is. Look up any entity types that you're not familiar with:

In [ ]:
spacy.explain("GPE")

The last example comes from a political news article, which is pretty typical for what NER is often trained on and used for. Let's look at another news article, this one with a business focus:

In [ ]:
# Example 2
text = """Taco Bell’s latest marketing venture, a pop-up hotel, opened at 10 a.m. Pacific Time Thursday. 
The rooms sold out within two minutes.
The resort has been called “The Bell: A Taco Bell Hotel and Resort.” It’s located in Palm Springs, California."""

In [ ]:
doc = nlp(text)

In [ ]:
displacy.render(doc, style="ent")

## Discussion
Compare how the NER performs on each of these texts. Can you see any errors? Why do you think it might make those errors?

## Coding Exercise
Write a function to that takes a doc as an argument and returns a dictionary mapping each entity type label to a list of that entity in the doc. Try creating a few different doc instances and testing this function out.

**Note**: A doc's entities can be accessed in the attribute `doc.ents`. An entity's label can be accessed in the attribute `ent.label_`.

In [ ]:
from collections import defaultdict

def collect_entities(doc):
    """
    """
    d = defaultdict(list)
    # Your code here
    return d

In [ ]:
collect_entities(doc)

# Clinical Text
Let's now try using spaCy's built-in NER model on clinical text.

In [ ]:
clin_text = "76 year old man with hypotension, CKD Stage 3, status post RIJ line placement and Swan.  "

In [ ]:
doc = nlp(clin_text)

In [ ]:
displacy.render(doc, style="ent")

**Discussion**
- How did spaCy do with this sentence?
- What do you think caused it to make errors in the classifications?

General purpose NER models are typically made for extracting entities out of news articles. As we saw before, this includes mainly people, organizations, and geopolitical entities. 

**Discussion**
- What are some entity types we are interested in in clinical domain?
- Does spaCy's out-of-the-box NER handle any of these types>

In [ ]:
ner = nlp.pipeline[-1][1]

In [ ]:
ner.labels

# Pattern Matching
TODO write this part later.
https://explosion.ai/demos/matcher

In [ ]:
from spacy.matcher import Matcher

In [ ]:
matcher = Matcher(nlp.vocab)

In [ ]:
doc = nlp(clin_text)

In [ ]:
doc

In [ ]:
patterns = [
    [{'TEXT': 'hypotension'}],
    [{'LOWER': 'CKD'}, {'TEXT': 'Stage'}, {'TEXT': '3'}],
]

In [ ]:
for pattern in patterns:
    print(pattern)
    matcher.add('CONDITION_PATTERN', None, pattern)

In [ ]:
matches = matcher(doc)
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

Let's look at a slightly different text now. Try to write one single pattern to capture "stage 4 ckd" and " Stage 3 CKD"

In [ ]:
clin_text2 = "the pt presents for stage 4 ckd. He previously had Stage 3 CKD."

In [ ]:
pattern = [
    {'LOWER': 'stage'},
    {'POS': 'NUM'},
    {'LOWER': 'ckd'},
]

In [ ]:
matcher.add('CONDITION_PATTERN', None, pattern)

In [ ]:
doc = nlp(clin_text2)

In [ ]:
matches = matcher(doc)
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

# EntityRuler

In [ ]:
from spacy.pipeline import EntityRuler

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes('ner')

In [ ]:
nlp.pipe_names

In [ ]:
ruler = EntityRuler(nlp)

In [ ]:
patterns = [
    {"label": "CONDITION", "pattern": [{'TEXT': 'stage'}, {'POS': 'NUM'}, {'LOWER': 'ckd'}]}
]

In [ ]:
ruler.add_patterns(patterns)
nlp.add_pipe(ruler, last=True, name='CONDITION_NER')

In [ ]:
nlp.pipe_names

In [ ]:
doc = nlp(clin_text2)

In [ ]:
displacy.render(doc, style="ent")

# Assignment: Write your own NER
Use the `EntityRuler` class to extract the following concepts from these texts:
- "PROCEDURE"
- "CONDITION"

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes('ner')

In [ ]:
long_text = (
    "There is continued mild-to-moderate congestive heart failure. "
    
    "87-year-old man with htn and end-stage renal disease. "
    
    "His wife recently died from end stage renal disease. "
    
    "The patient is s/p median sternotomy and right thoracotomy "
    
    "The pt presents for stage 4 ckd " 
    
    "He previously had stage 3 CKD."
    
    )

In [ ]:
long_text

In [ ]:
patterns = [
    # Your code here
    
]

In [ ]:
ruler = EntityRuler(nlp)

In [ ]:
ruler.add_patterns(patterns)
# nlp.add_pipe(ruler, last=True, name='CONDITION_NER')

In [ ]:
doc = nlp(long_text)

In [ ]:
options = {"colors": {"CONDITION": "#f5a742", 
                      "PROCEDURE": "#42f5e6"}}

In [ ]:
displacy.render(doc, style="ent", options=options)